In [3]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

from utility.indicator import Indicator
from utility.draw_chart import PlotChart
from get_kabuka import GetKabuka
from get_calendar import GetMarketCalendar
from database.models import KabuDatabase, CodeList, KabuPrice, MarketCalendar
from utility.indicator import Indicator

## Data作成

In [4]:
DB = KabuDatabase()
CL = CodeList()
KP = KabuPrice()
IND = Indicator()

list_name = DB.get_data('mylist')[0][1]
codes = DB.get_data(list_name)
dfs = []
for d in codes:
    col = ['index','date','open','high','low','close','volume','adjust_close']
    dfs.append(pd.DataFrame(np.array(KP.get_data('CODE'+str(d[1]))), columns = col))

df = IND.ohlc_with_indicator(dfs[0])

## Plot Data

In [5]:
def plot_candle_chart(df):
    return go.Candlestick(
                x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name = 'candle',
                increasing = dict(
                    line_width = 1,
                    line_color='red',
                    fillcolor='red',
                ),
                decreasing = dict(
                    line_width = 1,
                    line_color='blue',
                    fillcolor='blue'
                )
            )
def plot_indicator(df, indicator):
    if indicator == 'sma5':
        indicator_color='#ED7D32'
    elif indicator == 'sma25':
        indicator_color='#02CC96'
    elif indicator == 'sma50':
        indicator_color='#AB63F9'
    elif indicator == 'macd':
        indicator_color='#626DFA'
    elif indicator == 'macdsignal':
        indicator_color='#02CC96'
    elif indicator == 'rci':
        indicator_color='#626DFA'
    elif indicator == 'rsi9':
        indicator_color='#626DFA'
    elif indicator == 'rsi14':
        indicator_color='#02CC96'
    return go.Scatter(
            x=df['date'],
            y=df[indicator],
            name=indicator,
            marker_color=indicator_color
            )
def plot_bar_indicator(df, indicator):
    if indicator == 'macdhist':
        indicator_color='#ED7D32'
    return go.Bar(
            x=df['date'],
            y=df[indicator],
            name=indicator,
            marker=dict(
                color=indicator_color,
                line=dict(color='white', width=0.2)
            ))

# gyoseki bar chart
def plot_gyoseki_bar_chart():
    x = ['Q1','Q2','Q3','Q4']
    y = [10,20,30,40]
    return go.Bar(
        x=x,
        y=y,
        marker_color='#ED7D32'
    )
def plot_market_bar_chart():
    # x = np.arange(100)
    # y = np.random.randint(0,6,100)
    x = df['date']
    y = np.random.randint(0,6,len(df['date']))
    return go.Bar(
        x=x,
        y=y,
        marker_color='#ED7D32',
        name="importance"
    )
def plot_seiseki_pie_chart():
    labels = ['値上り銘柄','値下り銘柄','変わらず銘柄']
    values = [30, 10, 5]
    return go.Pie(
        labels=labels,
        values=values,
        hole=0.5,
        marker_colors=['#4471C4','#ED7D32','gray'],
    )
# karauri chart
def plot_karauri_chart():
    # x = np.arange(100)
    # y = np.random.randint(40,70,100)
    x = df['date']
    y = np.random.randint(40,70,len(df['date']))
    return go.Scatter(
        x=x,
        y=y,
        mode='lines',
        marker_color='#ED7D32'
    )

## plot by plotly

In [6]:
from plotly import tools
def create_figure(df):
    fig = tools.make_subplots(rows=4, cols=1,
                              vertical_spacing=0.0, 
                              shared_xaxes='all',
                              row_heights=[400,180,180,180])
    figure1= plot_candle_chart(df),
    figure2 = plot_indicator(df, 'sma5')
    figure3 = plot_indicator(df, 'sma25')
    figure4 = plot_indicator(df, 'sma50')
    figure5 = plot_indicator(df, 'rci')
    figure6 = plot_indicator(df, 'macd')
    figure7 = plot_indicator(df, 'macdsignal')
    figure8 = plot_bar_indicator(df, 'macdhist')
    figure9 = plot_indicator(df, 'rsi9')
    figure10 = plot_indicator(df, 'rsi14')
    colors = {
        "bgcolor":"#333F50",
        "plotbgcolor":"#232A34"
        }
    fig.add_trace(figure1[0], 1, 1)
    fig.add_trace(figure2, 1, 1)
    fig.add_trace(figure3, 1, 1)
    fig.add_trace(figure4, 1, 1)
    fig.add_trace(figure5, 2, 1)
    fig.add_trace(figure6, 3, 1)
    fig.add_trace(figure7, 3, 1)
    fig.add_trace(figure8, 3, 1)
    fig.add_trace(figure9, 4, 1)
    fig.add_trace(figure10, 4, 1)
    layout = dict(
        width=1000,
        xaxis_rangeslider_visible=False,
        xaxis = dict(
                showline=True, showgrid=True, gridwidth=0.1, gridcolor = 'gray', zeroline=False, showticklabels=False
            ),
        xaxis2 = dict(
                showline=True, showgrid=True, gridwidth=0.1, gridcolor = 'gray', zeroline=False, showticklabels=False
            ),
        xaxis3 = dict(
                showline=True, showgrid=True, gridwidth=0.1, gridcolor = 'gray', zeroline=False, showticklabels=False
            ),
        xaxis4 = dict(
                showline=True, showgrid=True, gridwidth=0.1, gridcolor = 'gray', zeroline=False, showticklabels=True, tickfont=dict(color='white')
            ),
        yaxis = dict(
            title='PRICE', titlefont=dict(color='white', size=17),showline=True, showgrid=True, gridwidth=0.1, gridcolor = 'gray', zeroline=True, showticklabels=True,
            tickfont=dict(color='white')
        ),
        yaxis2 = dict(
            title='RCI', titlefont=dict(color='white', size=17), showline=True, showgrid=False, zeroline=True, zerolinewidth=0.1, showticklabels=True,
            tickfont=dict(color='white'), range=(-100,100)
        ),
        yaxis3 = dict(
            title='MACD', titlefont=dict(color='white', size=17), showline=True, showgrid=False, zeroline=True, zerolinewidth=0.1, showticklabels=True,
            tickfont=dict(color='white'), range=(-100,100)
        ),
        yaxis4 = dict(
            title='RSI', titlefont=dict(color='white', size=17), showline=True, showgrid=False, zeroline=True, zerolinewidth=0.1, showticklabels=True,
            tickfont=dict(color='white'), range=(0,100)
        ),
        legend=dict(font=dict(color='white')),
        paper_bgcolor= colors["plotbgcolor"],
        plot_bgcolor= colors["plotbgcolor"],
    )
    fig['layout'].update(layout)
    fig.show()
    return fig

In [7]:
# ======= layout =========
def build_head():
    return html.H1(
            children='Non-Dimension.com',
            style={
                "margin-left":"0.5%",
                "margin-right":"0.5%",
                "margin-top":"0.5%",
                "margin-bottom":"0.5%",
                "padding":10,
                "textAlign":"center",
                "color":"white",
                "font":{"fontFamily":"sans-serif"},
                "fontSize":30,
                'background-color': colors["plotbgcolor"]
            })
# main area
def build_candle_chart_area():
    return html.Div(
            id = 'candle_chart',
            className = "nine columns",
            children=[dcc.Graph(
                figure=create_figure(df)
            )],
            style={
                "height":400,
                "overflowY": "scroll",
                "margin-left":"0.5%",
                "margin-right":"0%"
            }
        )
def build_gyoseki_bar_chart_area():
    return html.Div(
            id = 'bar_chart',
            className = "three columns",
            children=['build_gyoseki_bar_chart'
#                 draw_gyoseki_bar_chart()
            ],
            style={
                "height":400,
                "margin-left":"0.5%",
                "margin-right":"0%"
            }
        )
def build_chart_area():
    return html.Div(
            id="chart_area",
            className='row',
            children=[
                build_candle_chart_area(),
                build_gyoseki_bar_chart_area()
            ],
            style={
                "height":400,
                "margin-top":"0.5%"
                }
        )
# bottom area
def build_bottom_chart_area():
    return html.Div(
            id="bottom_chart_area",
            className='nine columns',
            children=[
                'draw_market_bar_chart()',
                'draw_karauri_chart()'
            ],
            style={
                "height":300,
                "margin-left":"0.5%",
                "margin-right":"0%"
            }
        )
def build_bottom_seiseki_area():
    return html.Div(
            id="bottom_seiseki_area",
            className='three columns',
            children=[
                'draw_seiseki_pie_chart()'
            ],
            style={
                "height":300,
                "margin-left":"0.5%",
                "margin-right":"0%"
            }
        )
def build_bottom_area():
    return html.Div(
            id="bottom_content_area",
            className='row',
            children=[
                build_bottom_chart_area(),
                build_bottom_seiseki_area()
            ],
            style={
                "height":300,
                "margin-top":"0.5%"
                }
            )

'''
----------------------------------------------
the following programm is for Web application
----------------------------------------------
'''
colors = {
    "bgcolor":"#333F50",
    "plotbgcolor":"#232A34"
    }
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    children=[
        build_head(),
        build_chart_area(),
        build_bottom_area()
    ],
    style = {
        'background-color': colors["bgcolor"]
        }
    )
# if __name__ == '__main__':
#     app.run_server(debug=True)

Exception: Invalid keyword argument: 'row_heights'

In [ ]:
# Layout
# def layout_candle_chart():
#     return go.Layout(
#         title='チャート',
#         xaxis_rangeslider_visible=False,
#         paper_bgcolor= colors["plotbgcolor"],
#         plot_bgcolor= colors["plotbgcolor"],
#         font = dict(color='white'),
#         xaxis = dict(
#             showline=True, showgrid=False, zeroline=False, showticklabels=False
#         ),
#         yaxis = dict(
#             showline=True, showgrid=False, zeroline=False
#         ),
#         margin=dict(l=50,r=50,b=10,t=50,pad=10),
#         autosize = True,
#         showlegend = False,
#     )
# def layout_indicator():
#     return go.Layout(
#         paper_bgcolor= colors["plotbgcolor"],
#         plot_bgcolor= colors["plotbgcolor"],
#         font = dict(color='white'),
#         xaxis = dict(
#             showline=True, showgrid=False, zeroline=False,
#         ),
#         yaxis = dict(
#             showline=True, showgrid=False, zeroline=False
#         ),
#         margin=dict(l=50,r=50,b=10,t=0,pad=10),
#         autosize = True,
#         showlegend = False,
#     )
# def layout_gyoseki_bar_chart():
#     return go.Layout(
#         title='四半期業績',
#         paper_bgcolor= colors["plotbgcolor"],
#         plot_bgcolor= colors["plotbgcolor"],
#         font = dict(color='white'),
#         margin=dict(l=50,r=50,b=10,t=50,pad=10),
#         autosize = True,
#         showlegend = False,
#     )
# seiseki pie chart
# def layout_seiseki_pie_chart():
#     return go.Layout(
#         title='成績',
#         paper_bgcolor= colors["plotbgcolor"],
#         plot_bgcolor= colors["plotbgcolor"],
#         font = dict(color='white'),
#         margin=dict(l=50,r=50,b=10,t=50,pad=10),
#         autosize = True,
#         showlegend = True,
#     )
# market calendar chart
# def layout_market_bar_chart():
#     return go.Layout(
#         title='経済指標重要度',
#         paper_bgcolor= colors["plotbgcolor"],
#         plot_bgcolor= colors["plotbgcolor"],
#         font = dict(color='white'),
#         autosize = True,
#         margin=dict(l=50,r=50,b=10,t=50,pad=10),
#         showlegend = False,
#     )
# def layout_karauri_chart():
#     return go.Layout(
#         title='空売り比率',
#         paper_bgcolor= colors["plotbgcolor"],
#         plot_bgcolor= colors["plotbgcolor"],
#         font = dict(color='white'),
#         autosize = True,
#         margin=dict(l=50,r=50,b=10,t=50,pad=10),
#         showlegend = False,
#     )

In [ ]:
# candle chart
# def draw_candle_chart_with_indicator():
#     return dcc.Graph(
#             figure=go.Figure(
#                 data=[
#                     plot_candle_chart(df),
#                     plot_indicator(df, 'sma5'),
#                     plot_indicator(df, 'sma25'),
#                     plot_indicator(df, 'sma50')
#                 ],
#                 layout=layout_candle_chart()
#             ),
#             style={
#                 "width":"100%",
#                 "height":300
#                 },
#             )
# def draw_indicator():
#     return dcc.Graph(
#             figure=go.Figure(
#                 data=[
#                     plot_indicator(df, 'rci'),
#                 ],
#                 layout=layout_indicator()
#             ),
#             style={
#                 "width":"100%",
#                 "height":120
#                 },
#             )
# def draw_gyoseki_bar_chart():
#     return dcc.Graph(
#             figure=go.Figure(
#                 data=[plot_gyoseki_bar_chart()],
#                 layout=layout_gyoseki_bar_chart()
#             ),
#             style={
#                 "width":"112%",
#                 "height":400}
#             )

# def draw_market_bar_chart():
#     return dcc.Graph(
#             figure=go.Figure(
#                 data=[plot_market_bar_chart()],
#                 layout=layout_market_bar_chart()
#             ),
#             style={
#                 "width":"100%",
#                 "height":150}
#             )

# def draw_karauri_chart():
#     return dcc.Graph(
#             figure=go.Figure(
#                 data=[plot_karauri_chart()],
#                 layout=layout_karauri_chart()
#             ),
#             style={
#                 "width":"100%",
#                 "height":150}
#             )

# def draw_seiseki_pie_chart():
#     return dcc.Graph(
#             figure=go.Figure(
#                 data=[plot_seiseki_pie_chart()],
#                 layout=layout_seiseki_pie_chart()
#             ),
#             style={
#                 "width":"112%",
#                 "height":300}
#             )